# Bout, Stim, and Duration Post Hoc Analysis(CSV for Database) 

In [1543]:
import joblib
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import pickle
import time
from tkinter import *
import tkinter as tk
from tkinter import filedialog
from tqdm import tqdm
from tqdm import notebook
import itertools
import math
import seaborn as sns
import statistics 
from collections import OrderedDict
import csv

In [1542]:
# Create Tk root
root = Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)
infiles = filedialog.askopenfilename(multiple=True, title='load posthoc-predictions.npy')

%gui tk

In [1544]:
infiles

''

In [1545]:
Random_List = [] #creates a array known as random list 
for f in range(len(infiles)):
    arr = np.load(infiles[f])
    Random_List.extend(arr) 

# Total Time of Video in deciseconds

In [1546]:
duration_in_deciseconds = len(Random_List)
duration_in_minutes = duration_in_deciseconds / 600
duration_in_frames = duration_in_deciseconds * 3

print('This video is', duration_in_deciseconds, 'deciseconds, which is', duration_in_minutes, 'minutes, which is', duration_in_frames, 'frames')
#print(Random_List)

This video is 0 deciseconds, which is 0.0 minutes, which is 0 frames


## Creating Array

## Time Integration (deciseconds) 

In [1547]:
BehaviorTime = []

for index, value in enumerate(Random_List):
    BehaviorTime.append((value, index ))

In [1548]:
#print(BehaviorTime)

### Ensures you only analyze expected length of  Session 

In [1549]:

length_of_session = (int(input("How long was the session in minutes?"))) * 600
print(length_of_session)
BehaviorTime = BehaviorTime[:(length_of_session)]

KeyboardInterrupt: Interrupted by user

#### Removes all info after selected end time 

In [ ]:
print('This video is ', len(BehaviorTime), ' deciseconds, which is', (len(BehaviorTime)/600), 'minutes' )
print()
print(BehaviorTime)

## Adds Stim Count and Frame Rate to Array 

In [ ]:
for z in BehaviorTime:
    behavior = z[0]
    time= z[1]
behavior_type_and_duration = []
current_behavior = BehaviorTime[0][0]
current_time = BehaviorTime[0][1]  # Initialize with the first time
count = 0
frame_rate = 0
start_time = 0

for behavior, time in BehaviorTime:
    if behavior == current_behavior:
        count += 1
        if count == 1:
            start_time= current_time
            current_time = time + 1 # Update the current time for the ongoing behavior
            frame_rate= count * 3
        else:
            current_time = time + 1 # Update the current time for the ongoing behavior
            frame_rate= count * 3
    else:
        behavior_type_and_duration.append((current_behavior, count,frame_rate, start_time, current_time))
        current_behavior = behavior
        current_time = time + 0.01  #Update the current time for new behavior
        start_time= current_time
        count = 1
        frame_rate= count * 3
        
# Append the final (current_behavior, current_time, count) tuple outside the loop
behavior_type_and_duration.append((current_behavior, count, frame_rate, start_time, current_time))

# Make a copy of the original behavior_type_and_duration list
behaviortdt= list(behavior_type_and_duration)

In [ ]:
#print(behaviortdt)
#print(len(behaviortdt))

## Add Bout Count to the Array 

In [ ]:
enumerated_behaviortdt = []
behavior_index = {}  # Dictionary to store behavior and its corresponding index

for behavior, stim_count, frame_rate, start_time, end_time in behaviortdt:
    if behavior not in behavior_index:
        behavior_index[behavior] = 1
    else:
        behavior_index[behavior] += 1
    
    bout = behavior_index[behavior]
    enumerated_behaviortdt.append((behavior, stim_count,frame_rate, bout, start_time,end_time))

print(enumerated_behaviortdt)

# Duration Calcultaion

In [ ]:
# Calculate durations and update enumerated_behaviortdt
BCBDT = []

for behavior, stim_count,frame_rate, bout, start_time, end_time in enumerated_behaviortdt:
    duration = (frame_rate * (1/30))*10
    BCBDT.append((behavior, stim_count, frame_rate, bout, duration,start_time, end_time))

print(BCBDT)
BCBDST=BCBDT

In [ ]:
behavior_labels = {
    0.0: "Left Turns",
    1.0: "Right Turns",
    2.0: "Locomotion",
    3.0: "Face Groom",
    4.0: "All Other Groom",
    5.0: "All Other",
}

bdbt_dict = {}  # Dictionary to store lists of behaviors

for behavior, stim_count, frame_rate, bout, duration, start_time,end_time in BCBDST:
    behavior_label = behavior_labels.get(behavior, "Unknown Behavior")
    if behavior_label not in bdbt_dict:
        bdbt_dict[behavior_label] = []
    bdbt_dict[behavior_label].append((behavior, stim_count, frame_rate, bout, duration, start_time,end_time))


In [ ]:
#print(bdbt_dict)

In [ ]:
keys_list = bdbt_dict.keys()
#print(keys_list)

## Stim Count and Average Duration 

In [ ]:
from tabulate import tabulate
stimcount_by_key = {}

def round_float(value):
    return round(value, 3)

for key, tuples_list in bdbt_dict.items():
    stimcount = [tuple_item[1] for tuple_item in tuples_list]
    sum_of_stimcount = sum(stimcount)
    stimcount_by_key[key] = sum_of_stimcount

table_data = [(key, round_float(sum_value)) for key, sum_value in stimcount_by_key.items()]
headers = ["Key", "Total Number of Stimulations"]

table = tabulate(table_data, headers, tablefmt="pretty")
print(table)

average_duration_by_key = {}

for key, tuples_list in bdbt_dict.items():
    durations = [tuple_item[4] for tuple_item in tuples_list]
    average_duration = ((sum(durations) / len(durations))/10)
    average_duration_by_key[key] = round_float(average_duration)

table_data = [(key, avg_duration) for key, avg_duration in average_duration_by_key.items()]
headers = ["Key", "Average Duration(seconds)"]

table = tabulate(table_data, headers, tablefmt="pretty")
print(table)

stim_block_stim_count_by_key = {}

for key, tuples_list in bdbt_dict.items():
    sum_of_stimcount = 0
    for tuple_item in tuples_list:
        if 3000 <= tuple_item[5] and tuple_item[6] <= 21000:  # Check the fourth element's value
            sum_of_stimcount += tuple_item[1]
    stim_block_stim_count_by_key[key] = sum_of_stimcount

table_data = [(key, round_float(sum_value)) for key, sum_value in stim_block_stim_count_by_key.items()]
headers = ["Key", "Total Stimulations during 30-Min Stim Block"]
#for (key, value) in stim_block_stim_count_by_key:
    #print(key)
    #print(value)
table = tabulate(table_data, headers, tablefmt="pretty")
print(table)

average_duration_30min_by_key = {}

for key, tuples_list in bdbt_dict.items():
    sum_of_duration = 0
    for tuple_item in tuples_list:
        if 3000 <= tuple_item[5] and tuple_item[6] <= 21000:  # Check the fourth element's value
            sum_of_duration += tuple_item[3]
    average_duration = ((sum_of_duration / (len(tuples_list)) )/ 10)  # Use len(tuples_list) instead of len(sum_of_stimcount)
    average_duration_30min_by_key[key] = round_float(average_duration)

table_data = [(key, avg_duration) for key, avg_duration in average_duration_30min_by_key.items()]
headers = ["Key", "Average Duration During 30 Min Stim Block(seconds)"]
    
table = tabulate(table_data, headers, tablefmt="pretty")
print(table)



##  Number of Bouts

In [ ]:
bout_by_key = {}

for key, tuples_list in bdbt_dict.items():
    bouts = [tuple_item[3] for tuple_item in tuples_list]
    sum_of_bouts = len(bouts)
    bout_by_key[key] = sum_of_bouts

table_data = [(key, sum_value) for key, sum_value in bout_by_key.items()]
headers = ["Key", "Total Bouts"]

table = tabulate(table_data, headers, tablefmt="pretty")
print(table)

stim_block_bout_count_by_key = {}

for key, tuples_list in bdbt_dict.items():
    bout_count = sum(1 for tuple_item in tuples_list if 3000 <= tuple_item[5] and tuple_item[6] <= 21000)
    stim_block_bout_count_by_key[key] = bout_count

table_data = [(key, count) for key, count in stim_block_bout_count_by_key.items()]
headers = ["Key", "30 minute Stim Block Bout Count"]

table = tabulate(table_data, headers, tablefmt="pretty")
print(table)

## Behavior of Interest

In [ ]:
behavior_mapping = {
    0: "Left Turn ",
    1: "Right Turn ",
    2: "Locomotion ",
    3: "Face Groom ",
    4: "All Other Groom ",
    5: "All Other "
}

print("Please select the behavior of interest:")
for key, value in behavior_mapping.items():
    print(f"{key}: {value}")

behavior_of_interest_1 = int(input("Enter the number corresponding to the behavior of interest: "))

behavior_name_2 = behavior_mapping.get(behavior_of_interest_1, "Unknown")

print("Selected behavior of interest:", behavior_name_2)


## Type of Fiber Connection

In [ ]:
selection = input("Please enter the type of fiber connection for the trial (B for Bilateral, I for Ipsilateral, C for Contralateral): ")

if selection == 'b':
    type_of_fiber_connection = 'Bilateral'
elif selection == 'i':
    type_of_fiber_connection = 'Ipsilateral'
elif selection == 'c':
    type_of_fiber_connection = 'Contralateral'
else:
    type_of_fiber_connection = 'Unknown'

print("Selected type of fiber connection:", type_of_fiber_connection)


## Session Type 

In [ ]:
trial_type = ""

sham_session = input("Is it a Sham session? (y/n): ")

if sham_session.lower() == "y":
    trial_type = "Sham Session"

## Creating CSV

In [ ]:
rounded_dict = {}

for key, value_list in bdbt_dict.items():
    rounded_values = [
        (
            round(v[0]),     # Rounding and converting to integer
            v[1], v[2], v[3], # Keeping other integer values as they are
            round(v[4]),     # Rounding and converting to integer
            round(v[5], 2),  # Rounding the sixth float value
            round(v[6], 2)   # Rounding the seventh float value
        )
        for v in value_list
    ]
    rounded_dict[key] = rounded_values

#print(rounded_dict)

bdbt_dict=rounded_dict
#print(bdbt_dict)

In [ ]:
from json import dumps
string_bdbt_dict=dumps(bdbt_dict)

In [ ]:
# Find the start index of the first '{' character
start_index = string_bdbt_dict.index('{')

# Find the end index of the last '}' character
end_index = string_bdbt_dict.rindex('}') + 1

# Extract the dictionary portion of the string
dict_str = string_bdbt_dict[start_index:end_index]

# Find the start indices of key-value pairs using '":'
key_value_end = []
for pos, char in enumerate(dict_str):
    if dict_str[pos:pos+3] == '": ':
        key_value_end.append(pos)
        
#print(key_value_end)      

# Initialize lists to store extracted keys and values
keys = []
values = []
value_ends=[]

# Iterate through the start indices of key-value pairs
for key_value_start in key_value_end:
    # Find the start index of the value by searching for '[[' after the key-value pair
    value_start = dict_str.find('[[', key_value_start) + 1
    
    # Find the end index of the value by searching for ']]' after the value_start
    value_end = dict_str.find(']]', value_start)+ 2
    value_ends.append(value_end)

    # Extract the value using the calculated start and end indices
    value = dict_str[value_start:value_end]
    values.append(value)
    # Extract the key using the portion of the string before the '":'
    if key_value_start == key_value_end[0]:
        key = dict_str[2:key_value_start].strip()
        keys.append(key)
    else:
        key = dict_str[value_ends[-2]+ 3:key_value_start].strip()
        keys.append(key)
        
#print(values)
#print(value_ends)
modified_values = {}  # Initialize an empty dictionary to store modified values

# Iterate through the keys and values
for key, value in zip(keys, values):
    modified_value = value.replace(" ", "").replace("],", "]").replace("]]","]")
    modified_values[key] = modified_value

In [ ]:
import os
import csv

keys_to_write = ["Right Turns", "Locomotion", "Face Groom"]

# Create a dictionary to hold cleaned and possibly split values
cleaned_values = {}

# Iterate through the keys and values, cleaning and splitting if needed
for key in keys_to_write:
    value = modified_values[key].strip()
    if len(value) > 32500:
        split_index = value.rfind("]", 0, 32500)  # Find the last "]" before 32500 characters
        if split_index != -1:  # If "]" is found before the limit
            cleaned_values[key] = value[:split_index + 1]  # Include the last "]"
            cleaned_values[f"{key} Continue"] = value[split_index + 1:]  # Continue from after the "]"
        else:
            # If "]" is not found, just store the value as it is
            cleaned_values[key] = value
    else:
        cleaned_values[key] = value

# Format specific values in cleaned_values dictionary
formatted_values = {'"{}"'.format(key): value for key, value in cleaned_values.items()}

    
# Specify the output path for the CSV file
output_path = r"Z:\KayCei\Database 1.csv"

data_dict = {}
session_numbers = {}

# Iterate through each file in infiles
for file_path in infiles: # if taking from Z Drive 
    folders = (os.path.dirname(file_path)).split('/')
    print(folders)
    # Extract the mouse name from the file path
    mouse_name = folders[2].split('-')[0]
    # Extract the genotype from the file path
    genotype = folders[2].split('-')[1]
    # Extract the date from the file path
    date = folders[-1].split('-')[0]
#for file_path in infiles:   #if taking files from D drive
    #folders = (os.path.dirname(file_path)).split('/')
    #print(folders)
    # Extract the mouse name from the file path
    #mouse_name = folders[6].split('-')[0]
    # Extract the genotype from the file path
    #genotype = folders[6].split('-')[1]
    # Extract the date from the file path
    #date = folders[-1].split('-')[0]    
# Determine the session number for this mouse and stim behavior
    key = (mouse_name, behavior_name_2, type_of_fiber_connection)
    if key not in session_numbers:
        session_numbers[key] = 1
    else:
        session_numbers[key] += 1
    session_number = session_numbers[key]

    # Retrieve the relevant statistics from the dictionaries
    right_turn_bdbt=cleaned_values.get("Right Turns",0)
    right_turn_bout_count = bout_by_key.get("Right Turns", 0)
    right_turn_stim_count = stimcount_by_key.get("Right Turns", 0)
    average_duration_right_turn = average_duration_by_key.get("Right Turns", 0)
    right_turn_bout_30min=stim_block_bout_count_by_key.get("Right Turns", 0)
    right_turn_stim_30min = stim_block_stim_count_by_key.get("Right Turns", 0)
    right_turn_duration_30min=average_duration_30min_by_key.get("Right Turns", 0)
    
    loc_bdbt = cleaned_values.get("Locomotion",0)
    loc_bout_count = bout_by_key.get("Locomotion", 0)
    loc_stim_count = stimcount_by_key.get("Locomotion",0)
    average_duration_loc = average_duration_by_key.get("Locomotion",0)
    loc_stim_30min = stim_block_stim_count_by_key.get("Locomotion",0)
    loc_bout_30min=stim_block_bout_count_by_key.get("Locomotion",0)
    loc_duration_30min=average_duration_30min_by_key.get("Locomotion",0)
    
    face_groom_bdbt = cleaned_values.get("Face Groom",0)
    face_groom_bout_count = bout_by_key.get("Face Groom", 0)
    face_groom_stim_count = stimcount_by_key.get("Face Groom", 0)
    average_duration_face_groom =average_duration_by_key.get("Face Groom", 0)
    face_groom_bout_30min=stim_block_bout_count_by_key.get("Face Groom", 0)
    face_groom_stim_30min = stim_block_stim_count_by_key.get("Face Groom", 0)
    face_groom_duration_30min=average_duration_30min_by_key.get("Face Groom", 0)
    
    # Apply cleaning and splitting logic to the arrays
    cleaned_right_turn_bdbt = right_turn_bdbt.strip()
    cleaned_loc_bdbt = loc_bdbt.strip()
    cleaned_face_groom_bdbt = face_groom_bdbt.strip()
    rt_array_cont=cleaned_values.get("Right Turns Continue",0)
    loc_cont=cleaned_values.get("Locomotion Continue",0)
    fg_cont=cleaned_values.get("Face Groom Continue",0)
    
    mouse_data = {
        "Mouse": mouse_name,
        "Genotype": genotype,
        "Date": date,
        "Selected Behavior": behavior_name_2,
        "Sham": sham_session,  
        "Session Number": session_number,
        "Fiber Connection": type_of_fiber_connection,
        "Right Turn Array": cleaned_right_turn_bdbt,
        "Right Turn Array Continue":rt_array_cont,
        "Total Right Turn Bout Count": right_turn_bout_count,
        "Total Right Turn Stim Count": right_turn_stim_count,
        "Average Duration of Right Turn": average_duration_right_turn,
        "Total Right Turn Stim in Stim Block": right_turn_stim_30min,
        "Total Right Turn Bouts in Stim Block":right_turn_bout_30min,
        "Right Turn Average Duration in Stim Block":right_turn_duration_30min,
        "Locomotion Array": cleaned_loc_bdbt,
        "Locomotion Array Continue": loc_cont,
        "Total Locomotion Bout Count": loc_bout_count,
        "Total Locomotion Stim Count": loc_stim_count,
        "Average Duration of Locomotion": average_duration_loc,
        "Total Locomotion Stim in Stim Block": loc_stim_30min,
        "Total Locomotion Bouts in Stim Block":loc_bout_30min,
        "Locomotion Average Duration in Stim Block":loc_duration_30min,
        "Face Groom Array": cleaned_face_groom_bdbt,
        "Face Groom Array Continue": fg_cont,
        "Total Face Groom Bout Count": face_groom_bout_count,
        "Total Face Groom Stim Count": face_groom_stim_count,
        "Average Duration of Face Groom": average_duration_face_groom,
        "Total Face Groom Stim in Stim Block": face_groom_stim_30min,
        "Total Face Groom Bouts in Stim Block":face_groom_bout_30min,
        "Face Groom Average Duration in Stim Block":face_groom_duration_30min,
    }

    data_dict[mouse_name] = mouse_data

# Define the headers for the CSV file
headers = ["Mouse", "Genotype", "Date", "Selected Behavior", "Sham", "Session Number", "Fiber Connection",
           "Right Turn Array","Right Turn Array Continue","Total Right Turn Bout Count", "Total Right Turn Stim Count", "Average Duration of Right Turn",
           "Total Right Turn Stim in Stim Block","Total Right Turn Bouts in Stim Block","Right Turn Average Duration in Stim Block",
           "Locomotion Array","Locomotion Array Continue","Total Locomotion Bout Count","Total Locomotion Stim Count", "Average Duration of Locomotion",
           "Total Locomotion Stim in Stim Block","Total Locomotion Bouts in Stim Block","Locomotion Average Duration in Stim Block",
           "Face Groom Array","Face Groom Array Continue","Total Face Groom Bout Count","Total Face Groom Stim Count","Average Duration of Face Groom",
           "Total Face Groom Stim in Stim Block","Total Face Groom Bouts in Stim Block","Face Groom Average Duration in Stim Block",
          ]

if os.path.exists(output_path):
    # Keep track of existing entries in the CSV to avoid duplicates
    existing_entries = set()
    entry_key2_session_numbers = {}
    sorted_entries = []

    # Read existing data from the CSV and add it to the set
    with open(output_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=';')
        for row in reader:
            entry_key = (row["Mouse"], row["Selected Behavior"], row["Fiber Connection"], row["Date"])
            entry_key2 = (row["Mouse"], row["Selected Behavior"], row["Fiber Connection"], row["Sham"])
            existing_entries.add(entry_key)
            entry_key2_session_numbers[entry_key2] = int(row["Session Number"])
            sorted_entries.append(row)
        
else:
    existing_entries = set()
    entry_key2_session_numbers = {}
    
# Ensure the output directory exists before writing the CSV file
output_dir = os.path.dirname(output_path)
os.makedirs(output_dir, exist_ok=True)

# Write the data from data_dict to the CSV file (in append mode 'a')
with open(output_path, 'a', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers, delimiter=';')

    # Check if the file is empty, if so, write the header
    if csvfile.tell() == 0:
        writer.writeheader()

for mouse_data in data_dict.values():
    entry_key = (mouse_data["Mouse"], mouse_data["Selected Behavior"], mouse_data["Fiber Connection"], mouse_data["Date"])
    entry_key2 = (mouse_data["Mouse"], mouse_data["Selected Behavior"], mouse_data["Fiber Connection"], mouse_data["Sham"])

    # Check if the entry already exists in the CSV
    if entry_key not in existing_entries:
        session_number = entry_key2_session_numbers.get(entry_key2, 0) + 1
        existing_entries.add(entry_key)
        entry_key2_session_numbers[entry_key2] = session_number

        for formatted_key in ["Right Turn Array", "Right Turn Array Continue", "Locomotion Array", "Locomotion Array Continue"]:
            if formatted_key in formatted_values:
                mouse_data[formatted_key] = formatted_values[formatted_key]

        # Write the complete mouse data to the CSV
        sorted_entries.append(mouse_data)  # Add the new entry

        print("Entry added Successfully:", entry_key)
    else:
        print("Entry already exists in the CSV:", entry_key)

# Sort the entries based on date within each group of entries that share the same entry_key2
sorted_entries.sort(key=lambda x: (x["Mouse"], x["Selected Behavior"], x["Fiber Connection"], x["Sham"], x["Date"]))

# Update the session numbers based on the sorted order
entry_key2_counter = {}  # To track the session numbers within each entry_key2 group

for entry in sorted_entries:
    entry_key2 = (entry["Mouse"], entry["Selected Behavior"], entry["Fiber Connection"], entry["Sham"])
    
    if entry_key2 not in entry_key2_counter:
        entry_key2_counter[entry_key2] = 1
    else:
        entry_key2_counter[entry_key2] += 1
    
    entry["Session Number"] = entry_key2_counter[entry_key2]
    
# Sort the entries based on date within each group of entries that share the same entry_key2
sorted_entries.sort(key=lambda x: (x["Mouse"], x["Selected Behavior"], x["Fiber Connection"], x["Sham"], x["Date"]))

# Rewrite the CSV file with sorted and formatted entries
with open(output_path, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers, delimiter=';')
    writer.writeheader()
    writer.writerows(sorted_entries)